ATS 2020

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from tqdm import tqdm_notebook

In [3]:
import requests

In [4]:
import time

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

In [23]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [11]:
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')

In [7]:
driver.get('https://www.abstractsonline.com/pp8/#!/8998/sessions/*/1')

In [25]:
ResultsButton = driver.find_element_by_xpath('//*[@id="filter"]/div/div/button[3]')

In [26]:
ResultsButton.click()

In [31]:
ids = BeautifulSoup(driver.page_source)

In [48]:
t =ids.find_all('h1',class_ = 'name')

In [ ]:
//*[@id="paginator"]/div/ul/li[7]/span

In [71]:
pg = driver.find_element_by_css_selector('#paginator > div > ul > li.next > span') #2 for page 1
pg.click()

- Get sessions in a dataframe/dictionary with session id as key along with objectives, descriptions and links of presentations, with a new row for each link
- Iterate through these rows and obtain information on each of the presentations.

Extract all SessionLinks

In [12]:
SessInfo = {}
IterationDict = {}
i = 0
driver.get('https://www.abstractsonline.com/pp8/#!/9045/sessions/*/1')
timeout = 500
locator = '//*[@id="filter"]/div/div/button[3]'
WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
time.sleep(1)
ResultsButton = driver.find_element_by_xpath('//*[@id="filter"]/div/div/button[3]')
ResultsButton.click()
prevurl = ''
currenturl = driver.current_url
time.sleep(1)
while prevurl!=currenturl:
    prevurl = currenturl
    print(currenturl)
    time.sleep(1)
    Data = BeautifulSoup(driver.page_source)
    Results = Data.find('ul',id = 'results')
#     print(Results)
    Sessions = Results.find_all('div',class_ = 'span9')
    for j in Sessions:
        
#         SessInfo['Title'] = j.find('span',class_ = 'title color-primary').text
#         SessInfo['url'] = 'https://www.abstractsonline.com/pp8/#!/8998/session/'+ j.find('h1',class_ = 'name')['data-id']
#         print(SessInfo)
        IterationDict[j.find('h1',class_ = 'name')['data-id']] = {}
        IterationDict[j.find('h1',class_ = 'name')['data-id']]['Title'] = j.find('span',class_ = 'title color-primary').text
        IterationDict[j.find('h1',class_ = 'name')['data-id']]['url'] = 'https://www.abstractsonline.com/pp8/#!/9045/session/'+ j.find('h1',class_ = 'name')['data-id']
        i+=1
    time.sleep(1)
    pg = driver.find_element_by_css_selector('#params > div:nth-child(1) > div > ul > li.next > span') #2 for page 1
    pg.click()
    currenturl = driver.current_url
#     print(currenturl)

https://www.abstractsonline.com/pp8/#!/9045/sessions/*/1


In [10]:
driver.close()

In [8]:
len(IterationDict.keys())

29

In [13]:
IterationDict


{'838': {'Title': ' VMS.MCB09.01. Evaluating Cancer Genomics from Normal Tissues through Evolution to Metastatic Disease',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/838'},
 '843': {'Title': ' VPO.CT02. Phase II Clinical Trials',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/843'},
 '841': {'Title': ' VCTPL01. Opening Clinical Plenary',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/841'},
 '834': {'Title': ' VMS.CL11.01. Predictive Biomarkers for Immunotherapeutics',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/834'},
 '853': {'Title': ' VCTPL04. Immunotherapy Clinical Trials 2',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/853'},
 '842': {'Title': ' VPO.CT01. Phase I Clinical Trials',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/842'},
 '844': {'Title': ' VPO.CT03. Phase III Clinical Trials',
  'url': 'https://www.abstractsonline.com/pp8/#!/9045/session/844'},
 '848': {'Title': ' VCTP

Extract SessionDetails and urls for presentations

In [14]:
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')

In [14]:
SessionDetails = {}

for _id in tqdm_notebook(IterationDict.keys()):
    iteration = 1
#     driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')
    url = IterationDict[_id]['url']
    driver.get(url)
#     time.sleep(1)
    timeout = 500
    locator = '//*[@id="body"]/div/div[2]/div[2]/table'
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
    Data = BeautifulSoup(driver.page_source)
    Table = Data.find('div',class_ = 'span7 presentation-table')
    trs = Table.find('table',class_ = 'table').find_all('tr')
    try:
        Objectives = Data.find('div',class_ = 'span3 offset1').text
    except:
        Objectives = ""
    try:
        Description = Data.find('blockquote',class_ = 'description').text
    except:
        Description = ""
    Location = Data.find('table',class_ = 'table table-bordered session-meta')
    Date = Location.find_all('td')[0].text
    Venue = Location.find_all('td')[1].text
    for tr in trs:
        t = _id + '-' + str(iteration)
        SessionDetails[t] = {}
        SessionDetails[t]['Session Title'] = IterationDict[_id]['Title']
        SessionDetails[t]['Description'] = Description
        SessionDetails[t]['Objectives'] = Objectives
        SessionDetails[t]['PresentationUrl'] = 'https://www.abstractsonline.com/pp8/'+tr.find_all('td')[1].find('a')['href']
        SessionDetails[t]['Date'] = Date
        #SessionDetails[t]['Location'] = Location
        iteration+=1
#     driver.close()

C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
SessionDetails

{'838-1': {'Session Title': ' VMS.MCB09.01. Evaluating Cancer Genomics from Normal Tissues through Evolution to Metastatic Disease',
  'Description': '',
  'Objectives': '\n\n\nSession Type\nVirtual Minisymposium\nTrack(s)\nMolecular and Cellular Biology/Genetics\n\n\n',
  'PresentationUrl': 'https://www.abstractsonline.com/pp8/#!/9045/presentation/10564',
  'Date': ' April 28, 2020, 3:50 PM - 5:35 PM'},
 '838-2': {'Session Title': ' VMS.MCB09.01. Evaluating Cancer Genomics from Normal Tissues through Evolution to Metastatic Disease',
  'Description': '',
  'Objectives': '\n\n\nSession Type\nVirtual Minisymposium\nTrack(s)\nMolecular and Cellular Biology/Genetics\n\n\n',
  'PresentationUrl': 'https://www.abstractsonline.com/pp8/#!/9045/presentation/10565',
  'Date': ' April 28, 2020, 3:50 PM - 5:35 PM'},
 '838-3': {'Session Title': ' VMS.MCB09.01. Evaluating Cancer Genomics from Normal Tissues through Evolution to Metastatic Disease',
  'Description': '',
  'Objectives': '\n\n\nSession

In [18]:
PresentationDetails = {}


for i in tqdm_notebook(SessionDetails.keys()):
    PresentationDetails[i]= {}
    PresentationDetails[i]['Session Title'] = SessionDetails[i]['Session Title'] 
    PresentationDetails[i]['Description'] = SessionDetails[i]['Description'] 
    PresentationDetails[i]['Objectives'] = SessionDetails[i]['Objectives']
    PresentationDetails[i]['PresentationUrl'] = SessionDetails[i]['PresentationUrl'] 
    PresentationDetails[i]['Date'] = SessionDetails[i]['Date']
#     PresentationDetails[i]['Location']  = SessionDetails[i]['Location'] 
    url = SessionDetails[i]['PresentationUrl']

#     url = SessionDetails[i]['PresentationUrl']
    driver.get(url)
#         driver.refresh()
    timeout = 500
    locator = '//*[@id="body"]/div/div[2]/div/dl/dd[2]'
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
    time.sleep(2.5)
    Data = BeautifulSoup(driver.page_source,'lxml')
    PresentationDetails[i]['Presentation Title'] = Data.find('h1',class_ ='color-primary').text
    PresentationDetails[i]['Presentation Time'] = Data.find('span',class_ = 'session-date').text
    PresentationDetails[i]['Presentation Location'] = Data.find('span',class_ = 'session-location').text
    ParticipantAbstract = Data.select('#body > div > div:nth-child(3) > div > dl')
    headers = ParticipantAbstract[0].find_all('dt')
    content = ParticipantAbstract[0].find_all('dd')
    for j in range(len(headers)):
        PresentationDetails[i][headers[j].text] = content[j].text
        

C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [300]:
url


'https://www.abstractsonline.com/pp8/#!/8998/presentation/10877'

In [301]:
driver.current_url == url

True

In [19]:
PresentationDetails

{'838-1': {'Session Title': ' VMS.MCB09.01. Evaluating Cancer Genomics from Normal Tissues through Evolution to Metastatic Disease',
  'Description': '',
  'Objectives': '\n\n\nSession Type\nVirtual Minisymposium\nTrack(s)\nMolecular and Cellular Biology/Genetics\n\n\n',
  'PresentationUrl': 'https://www.abstractsonline.com/pp8/#!/9045/presentation/10564',
  'Date': ' April 28, 2020, 3:50 PM - 5:35 PM',
  'Presentation Title': '\n                        \n\n                        \n                        \n                        \n                         - Chairperson\n                        \n\n\n                    ',
  'Presentation Time': ' April 28, 2020, 3:50 PM - 5:35 PM',
  'Presentation Location': '\n Virtual Meeting\n                        ',
  'Presenter/Authors': 'Samra Turajlic. The Francis Crick Institute, London, United Kingdom',
  'Abstract': 'There is no abstract associated with this presentation.'},
 '838-2': {'Session Title': ' VMS.MCB09.01. Evaluating Cancer G

In [199]:
t = list(SessionDetails.keys())
t[1973]

'107-9'

In [20]:
df = pd.DataFrame.from_dict(PresentationDetails,orient = 'index')

In [21]:
df

,Session Title,Description,Objectives,PresentationUrl,Date,Presentation Title,Presentation Time,Presentation Location,Presenter/Authors,Abstract,Disclosures
838-1,VMS.MCB09.01. Evaluating Cancer Genomics from...,,\n\n\nSession Type\nVirtual Minisymposium\nTra...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 3:50 PM - 5:35 PM",\n \n\n ...,"April 28, 2020, 3:50 PM - 5:35 PM",\n Virtual Meeting\n,"Samra Turajlic. The Francis Crick Institute, L...",There is no abstract associated with this pres...,NaN
838-2,VMS.MCB09.01. Evaluating Cancer Genomics from...,,\n\n\nSession Type\nVirtual Minisymposium\nTra...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 3:50 PM - 5:35 PM",\n \n\n ...,"April 28, 2020, 3:50 PM - 5:35 PM",\n Virtual Meeting\n,Elana Judith Fertig. Johns Hopkins Sidney Kimm...,There is no abstract associated with this pres...,NaN
838-3,VMS.MCB09.01. Evaluating Cancer Genomics from...,,\n\n\nSession Type\nVirtual Minisymposium\nTra...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 3:50 PM - 5:35 PM",\n \n\n ...,"April 28, 2020, 3:50 PM - 3:55 PM",\n Virtual Meeting\n,"Samra Turajlic. The Francis Crick Institute, L...",Abstract embargoed at this time.,NaN
838-4,VMS.MCB09.01. Evaluating Cancer Genomics from...,,\n\n\nSession Type\nVirtual Minisymposium\nTra...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 3:50 PM - 5:35 PM",\n \n\n ...,"April 28, 2020, 3:55 PM - 4:05 PM",\n Virtual Meeting\n,"Tapsi Kumar1, Quy Nguyen2, Kevin Nee2, Shansha...",Abstract embargoed at this time.,T. Kumar: None. Q. Nguyen: None. K. Nee: None...
838-5,VMS.MCB09.01. Evaluating Cancer Genomics from...,,\n\n\nSession Type\nVirtual Minisymposium\nTra...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 3:50 PM - 5:35 PM",\n \n\n ...,"April 28, 2020, 4:05 PM - 4:10 PM",\n Virtual Meeting\n,,There is no abstract associated with this pres...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
862-8,VCTPL09. COVID-19 and Cancer,\nDescription\nAccess to AACR Virtual Annual M...,\n\n\nSession Type\nVirtual Plenary Session\nT...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 9:00 AM - 10:45 AM",\n \n\n ...,"April 28, 2020, 10:30 AM - 10:45 AM",\n Virtual Meeting\n,,There is no abstract associated with this pres...,NaN
36-1,VSY.DDT03. New Drugs on the Horizon: Part 3,\nDescription\nThe Chemistry in Cancer Researc...,\n\n\nSession Type\nVirtual Symposium\nTrack(s...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 12:30 PM - 1:25 PM",\n \n\n ...,"April 28, 2020, 12:30 PM - 1:25 PM",\n Virtual Meeting\n,"Andrew J. Phillips. C4 Therapeutics, Watertown...",There is no abstract associated with this pres...,NaN
36-2,VSY.DDT03. New Drugs on the Horizon: Part 3,\nDescription\nThe Chemistry in Cancer Researc...,\n\n\nSession Type\nVirtual Symposium\nTrack(s...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 12:30 PM - 1:25 PM",\n \n\n ...,"April 28, 2020, 12:30 PM - 1:25 PM",\n Virtual Meeting\n,"Xiaojing Wang. Genentech, Inc., South San Fran...",There is no abstract associated with this pres...,NaN
36-3,VSY.DDT03. New Drugs on the Horizon: Part 3,\nDescription\nThe Chemistry in Cancer Researc...,\n\n\nSession Type\nVirtual Symposium\nTrack(s...,https://www.abstractsonline.com/pp8/#!/9045/pr...,"April 28, 2020, 12:30 PM - 1:25 PM",\n \n\n ...,"April 28, 2020, 12:35 PM - 12:50 PM",\n Virtual Meeting\n,"Edward B. Reilly. AbbVie Inc., North Chicago, ...",Abstract embargoed at this time.,"E. B. Reilly, AbbVie Inc Full-time/Part-time..."


In [22]:
df['Description'] = df['Description'].str.replace('\n','')

In [23]:
df['Objectives'] = df['Objectives'].str.replace('\n','')
df['Presentation Title'] = df['Presentation Title'].str.replace('\n','').str.strip()
df['Presentation Location'] = df['Presentation Location'].str.replace('\n','')

In [24]:
df['Presentation Start Time'] = df['Presentation Time'].str.split(',',expand = True)[2].str.split(' - ',expand = True)[0]
df['Presentation End Time'] =  df['Presentation Time'].str.split(',',expand = True)[2].str.split(' - ',expand = True)[1]

In [25]:
df['Presentation Date'] = df['Presentation Time'].str.split(',',expand = True)[0]+', '+df['Presentation Time'].str.split(',',expand = True)[1]

In [26]:
df.to_excel('AACR2020.xls')